# GANs

This is not so much as an execise, but rather a little example of producing a Gnerative Adverserial Network (GAN). 
Some of the code is a bit overly complicated, mainly because the output is saved as images. 

Overall, the GAN uses two parts - an generator and a discriminator. 


### Imports

Some imports for handling data, learning, and plotting

Please ignore any warnings

In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

from sklearn import preprocessing
import os,os.path
from functools import reduce

# Keras modules
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam,RMSprop

import seaborn
seaborn.set(style="ticks", color_codes=True)



### The simple GAN object

I encoded the method into a python class. Please note, that comments are sparse and I do appologise for it. 
For the sake of trying it out, please just accept the code as is. If you want to dive in, please let me know. 


In [ ]:
class simple_gan():

    def __init__(self, input_shape, output_shape, 
                 generator_input_dim,
                 optimizer=None,
                 metrics=['accuracy'],
                 deep_generator_shape=[8,8]):

        # check if optimizer has been supplied
        if not optimizer:
            # use default optimizer RMSProp
            self.optimizer = RMSprop()
        else:
            self.optimser  = optimizer

        self.metrics       = metrics
        self.ploted        = None
            
        self.input_shape           = input_shape             # input shape of the original data
        self.output_shape          = output_shape            # don't know yet
        self.generator_input_dim   = generator_input_dim     # shape of random vector
        self.deep_generator_shape  = deep_generator_shape
        
        # Build models
        self.generator_model     = self.build_generator_model(deep_generator_shape=deep_generator_shape)
        self.discriminator_model = self.build_discriminator_model()
        self.gan                 = self.build_and_compile_gan()

    # creates random samples using the generator part
    def generate_samples(self,number_of_samples=500):
        # generate artificial data
        if self.generator_model:
            random_vectors    = self.produce_random_vectors(number_of_samples)
            genenerated_batch = self.generator_model.predict(random_vectors)
            return genenerated_batch
            # return self.transformer.inverse_transform(genenerated_batch)
        return None


    
    def train(self, epochs, train_data, batch_size, 
              visulisation={'image_folder':None,'number_randon_vector':500},
              epoch_performance_display=10):

        number_randon_vector = 500
        image_folder         = None
        if visulisation:
            # save image_folder if passed as argument in visulisation - otherwise it stays None
            try: 
                image_folder  = visulisation['image_folder']
            except:
                pass
            number_randon_vector = visulisation['number_randon_vector']
        
        # for debugging
        epoch_performance_image_save = 100
        epoch_performance_quotient   = round(epoch_performance_image_save/epoch_performance_display)*epoch_performance_display

        train_data_scaled = train_data
        
        self.train_data_scaled = train_data_scaled
        
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        history = []
        self.random_vectors    = self.produce_random_vectors(number_randon_vector)
        self.genenerated_batch = self.train_data_scaled[:len(self.random_vectors)]
        if visulisation:
            self.plot_data(0,self.train_data_scaled,self.genenerated_batch,image_folder)

        
        for epoch in range(epochs):

            # Train Discriminator
            # select random training examples for batch
            batch_indexes = np.random.randint(0, train_data_scaled.shape[0], batch_size)
            training_batch = train_data_scaled[batch_indexes]
            
            # generate artificial data
            random_vectors    = self.produce_random_vectors(batch_size)
            genenerated_batch = self.generator_model.predict(random_vectors)

            # calculate loss
            loss_real = self.discriminator_model.train_on_batch(training_batch, real)
            loss_fake = self.discriminator_model.train_on_batch(genenerated_batch, fake)

            # calculate mean loss
            discriminator_loss = 0.5 * np.add(loss_real, loss_fake)

            #  Train Generator
            random_vectors    = self.produce_random_vectors(batch_size)
            generator_loss    = self.gan.train_on_batch(random_vectors, real)

            # keep history for plotting later
            history.append({"D":discriminator_loss[0],"G":generator_loss})


            # Plot the progress each epoch_performance_display time
            if epoch % epoch_performance_display == 0:
                print ("---------------------------------------------------------")
                print ("******************Epoch {}***************************".format(epoch))
                print ("Discriminator loss: {} ({},{})".format(discriminator_loss[0],loss_real,loss_fake))
                print ("Generator loss: {}".format(generator_loss))
                # print ('Noise epoch {}: <<{}>>'.format(epoch,random_vectors[0]))
                print ("---------------------------------------------------------")
                self.genenerated_batch = self.generator_model.predict(self.random_vectors)
                if visulisation:
                    if epoch % epoch_performance_quotient == 0:
                        self.plot_data(epoch,train_data_scaled,genenerated_batch,image_folder)
                    else:
                        self.plot_data(epoch,train_data_scaled,genenerated_batch,None)
        
        # plots loss if requested
        if visulisation:
            self.plot_loss(history,image_folder)
        else:
            self.plot_loss(history)
        plt.show()         

    def prepare_plot(self):
        # definitions for the axes
        left, width = -1, 3
        bottom, height = -1, 3
        bottom_h = left_h = left + width + 0.02
        rect_scatter = [left, bottom, width, height]
        
        fig = plt.figure(1, figsize=(8, 8))
        axScatter = plt.axes(rect_scatter)
        
        
        
    def plot_data(self,epoch,train_data,generated_data,image_folder=None):
        
        data_x = train_data[:,0].tolist()+generated_data[:,0].tolist()
        data_y = train_data[:,1].tolist()+generated_data[:,1].tolist()
        data_c = ['R']*len(train_data[:,0])+['P']*len(generated_data[:,0])
        
        df = pd.DataFrame(dict(data_x=data_x, data_y=data_y, data_c=data_c))
        
        if not self.ploted:
            self.prepare_plot()
            axes = plt.gca()
            axes.set_xlim(-1, +1)
            axes.set_ylim(-1, +1)
            self.ploted = seaborn.scatterplot('data_x', 'data_y', data=df, hue='data_c',legend=False)

            plt.ion()
            plt.show()
        else:
            plt.clf()
            axes = plt.gca()
            axes.set_xlim(-1, +1)
            axes.set_ylim(-1, +1)

            self.ploted = seaborn.scatterplot('data_x', 'data_y', data=df, hue='data_c',legend=False)
            plt.text(-0.9,0.9,str.zfill('{}'.format(epoch),6),bbox={'facecolor':'plum','pad':5,'edgecolor':'black'})
            plt.draw()
            plt.pause(0.001)


            
        if image_folder != None:
            if not os.path.exists(image_folder):
                os.mkdir(image_folder)
            plt.savefig('{}/perform_{}.png'.format(image_folder,str.zfill('{}'.format(epoch),6)))

               
            
        
    def plot_loss(self, history,image_folder=None):
        hist = pd.DataFrame(history)
        plt.figure(figsize=(20,5))
        for colnm in hist.columns:
            plt.plot(hist[colnm],label=colnm)
        plt.legend()
        plt.ylabel("loss")
        plt.xlabel("epochs")
        if image_folder != None:
            if not os.path.exists(image_folder):
                os.mkdir(image_folder)
            plt.savefig('{}/errors.png'.format(image_folder))
        plt.show()

        
    def produce_random_vectors(self, size):
        noise = np.random.normal(0, 1, (size, self.generator_input_dim))
        return noise
        # return self.generator_model.predict(noise)
        
    def build_generator_model(self,deep_generator_shape=[8,8]):
        generator_input   = Input(shape=(self.generator_input_dim,))

        deep_sequence_begin   = [Dense(deep_generator_shape[0], input_dim=self.generator_input_dim),LeakyReLU(alpha=0.2),]
        deep_sequence_middle  = reduce((lambda x, y: x + y), [[Dense(num_nodes),LeakyReLU(alpha=0.2),BatchNormalization(momentum=0.2)] for num_nodes in deep_generator_shape[1:]])
        deep_sequence_end     = [Dense(np.prod(self.input_shape), activation='tanh'),Reshape(self.input_shape)]
        deep_sequence         = deep_sequence_begin+deep_sequence_middle+deep_sequence_end
        generator_seqence     = Sequential(deep_sequence)

        generator_output_tensor = generator_seqence(generator_input)       
        generator_model = Model(generator_input, generator_output_tensor)

        return generator_model


    def build_discriminator_model(self):
        discriminator_input = Input(shape=self.input_shape)
        discriminator_sequence = Sequential([
            Flatten(input_shape=self.input_shape),
            # Dense(8,activation='tanh'),
            Dense(8),
            LeakyReLU(alpha=0.2),
            Dense(8),
            LeakyReLU(alpha=0.2),
            Dense(2),
            Dense(1)
            ])
    
        discriminator_output_tensor = discriminator_sequence(discriminator_input)
        discriminator_model         = Model(discriminator_input, discriminator_output_tensor)
        return discriminator_model

    def build_and_compile_gan(self):
        real_input = Input(shape=(self.generator_input_dim,))

        generator_output = self.generator_model(real_input)

        # compile the discriminator model
        self.discriminator_model.compile(loss='binary_crossentropy',optimizer=self.optimizer,metrics=self.metrics)
        self.discriminator_model.trainable = False
        
        discriminator_output = self.discriminator_model(generator_output)        
        
        gan = Model(real_input, discriminator_output)
        gan.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        return gan
        
    
    def compile_models(self):
        self.discriminator_model.compile(loss='binary_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
        self.discriminator_model.trainable = False

        
    def __str__(self):
        return_string = '{}'.format(self.generator_model)




## Creating artifical data

We start by creating artifical data following the simple function 

$f(x) = (x \cdot \epsilon) ^ 2$

Where $\epsilon$ is a random error for each example.


In [ ]:

percentage_error   = 0.5
input_values     = np.random.normal(0, 1, (100, 1))
input_data       = [[x[0],np.square(x[0])*(1+np.random.rand()*percentage_error)] for x in input_values]

input_data_train = np.array(input_data)
input_data_shape = input_data_train[0].shape
    
        

### Training and plotting the GAN

Next we set up the GAN and use 500 generated examples during training. 
The GAN will run for 2000 epochs - each time generating new data and trying to learn the discriminator. 

The construct 'visulisation_details' can also take in an 'image_folder'. If this information is given, the resulting plots are saved in this folder.


In [ ]:
simple_g = simple_gan(input_data_shape,(2,1),10,deep_generator_shape=[8,8])
visulisation_details={'number_randon_vector':500}
#visulisation_details={'image_folder':'./examples','number_randon_vector':500}
simple_g.train(2000,input_data_train,batch_size=256,visulisation=visulisation_details)

